In [5]:
import pandas as pd
import numpy as np
import random

# Read Data

In [6]:
# path = 'C:/Users/robin.opdam/Google Drive/Thesis (Msc)/Thesis_shared_files/'
path = '/Users/Robin/Google Drive/Thesis (Msc)/Thesis_shared_files/'

## Amazon Fashion

In [7]:
# data_path = 'data/Amazon/'
# file_name = 'Amazon_full' 
# file_name = 'Amazon_05_users' 
# file_name = 'Amazon_01_users'
# file_name = 'Amazon_005_users'
# file_name = 'Amazon_001_users'

## MovieLens

In [24]:
# data_path = 'data/ML/'
# file_name = 'ML_full' 
# file_name = 'ML_05_users'
# file_name = 'ML_01_users'
# file_name = 'ML_005_users'
# file_name = 'ML_001_users'

In [9]:
df = pd.read_pickle(path + data_path + file_name)
df.head()

,user,item,rating,datetime,item_id,user_id
21225350,137924,318,4.5,2017-09-20 19:20:13,314,137923
21225354,137924,527,3.5,2017-09-20 19:20:17,522,137923
21225351,137924,356,5.0,2017-09-20 19:20:20,351,137923
21225362,137924,1704,4.0,2017-09-20 19:20:22,1640,137923
21225367,137924,3147,4.0,2017-09-20 19:20:25,3054,137923


In [17]:
names_am = ['Amazon_05_users', 'Amazon_01_users', 'Amazon_005_users', 'Amazon_001_users']
names_ml = ['ML_05_users', 'ML_01_users', 'ML_005_users', 'ML_001_users']

# Data Prep
Create new ids for users and items that match the row and column indices of the user-item interaction matrix

In [18]:
df['item_id'] = df.item.astype('category').cat.codes
df['user_id'] = df.user.astype('category').cat.codes

## Leave item train test split

In [38]:
def leave_x_out(full_data, leave_out):
    # Input: data must be formatted by func: tranfsorm
    # Output: full_data = without all entries in leave one out set
    #         leave_one_out_set = data with one user and one item from full_data
    
    full_data['index'] = full_data.index
    user_items_ind = full_data.groupby('user_id')['index'].apply(list)
    index_to_drop = []
    
    for indices in user_items_ind:
        if len(indices) > leave_out:
            for to_leave_out in range(leave_out):
                index = indices[- to_leave_out]
                index_to_drop.append(index)
    
    leave_one_out_set = full_data.loc[index_to_drop]
    full_data_leave_one_out = full_data.drop(index_to_drop)
    
    return full_data_leave_one_out.drop(columns=['index']), leave_one_out_set.drop(columns=['index'])

In [39]:
def create_matrices(data, n_users, n_items):
        r = data['user_id']
        c = data['item_id']
        d = data['rating']
        m = sparse.csr_matrix((d, (r, c)), shape=(n_users, n_items))
        m_ones = m.copy()
        m_ones[m_ones > 0] = 1
                               
        return m, m_ones

# Random Benchmark

## Random user item dict

In [40]:
def create_random_ui_dict(test_set, rank_at=20):
    users = test_set.user_id.unique()
    items = test_set.item_id.unique()

    user_item_dict = {}

    random.shuffle(items)
    for u in users:
        item_index = random.randint(rank_at, len(items))
        user_item_dict[u] = items[item_index-rank_at:item_index]        

    return user_item_dict

In [41]:
def random_bench(file_names, path, steps, ranks_at, results):
    for name in file_names:
        print(name)
        df = pd.read_pickle(path + name)
        df['item_id'] = df.item.astype('category').cat.codes
        df['user_id'] = df.user.astype('category').cat.codes
        # Uses leave items out, but need leave users out?
        train_set, test_set = leave_x_out(df, 2)
        val_set, test_set = leave_x_out(test_set, 1)

        user_items = test_set.groupby('user_id')['item_id'].apply(list)
        hits = np.zeros((1,5))
        iterations = 10

        for iteration in range(iterations):
            ui_rand_dict = create_random_ui_dict(test_set)
            hitcounts = []
            recs_at = []
            precs_at = []
            for rank in ranks_at:
                hitcount = 0
                for u in test_set.user_id.unique():
                    for item in user_items[u]:
                        if item in ui_rand_dict[u][:rank]: #for 1 item test sets only
                            hitcount += 1

                hitcounts.append(hitcount)                     
            hits += np.array(hitcounts).T

        avg_hits = hits / iterations
        random_results[name] = avg_hits[0]
        print(avg_hits)
        
    return results

In [42]:
steps = 5
rank_at = 20
ranks_at = [1] + [i for i in range(steps, rank_at + steps, steps)]
random_results = pd.DataFrame(columns=names_am)
file_path = path + 'data/Amazon/'

random_results = random_bench(names_am, file_path, steps, ranks_at, random_results)

Amazon_05_users


KeyboardInterrupt: 

In [ ]:
pd.read_pickle('Results/BPR/rand_rank_hits')

In [ ]:
random_results['rank_at'] = ranks_at

In [26]:
random_results.to_pickle('Results/BPR/rand_rank_hits')

# Popularity Benchmark SHOULD BE COUNTS OF TRAIN SET
Popularity decides rank of item for everyone

In [27]:
file_names

['amazon_csj_2m',
 'df_amazon_csj_with_styles_0.63m_u_above_5_rui',
 '2m-ml',
 'ml_0.7_u_above_5',
 'ml_0.7_u_above_5_3_r_thres',
 '2m-ml_3_r_thres']

In [29]:
max_rank_at = 20
steps = 5
ranks_at = [1] + [i for i in range(steps, rank_at + steps, steps)]
items_in_test_set = 1
pop_results = pd.DataFrame(columns=file_names)

for name in file_names:
    print('\n', name)
    df = pd.read_pickle('Data/' + name)
    df_new_ids = transform(df)
    df_new_ids['item_counts'] = df_new_ids.groupby('item_id')['user_id'].transform('count') #for populairty
    train_set, test_set = leave_x_out(df_new_ids, items_in_test_set*2)
    val_set, test_set = leave_x_out(test_set, items_in_test_set)
    
    most_pop_items = test_set.sort_values('item_counts')['item_id'].unique()[-max_rank_at:]
    user_items = test_set.groupby('user_id')['item_id'].apply(list)
    
    hitcounts = []
    for rank in ranks_at:
        hitcount = 0
        for u in test_set.user_id.unique():
            for item in user_items[u]:
                if item in most_pop_items[:rank]:
                    hitcount += 1
        print('rank_at', rank, ' hitcount:', hitcount)
        hitcounts.append(hitcount)
    
    pop_results[name] = hitcounts


 amazon_csj_2m
Full data #row:  2059552
rank_at 1  hitcount: 15
rank_at 5  hitcount: 1390
rank_at 10  hitcount: 5840
rank_at 15  hitcount: 6460
rank_at 20  hitcount: 10185

 df_amazon_csj_with_styles_0.63m_u_above_5_rui
Full data #row:  629889
rank_at 1  hitcount: 23
rank_at 5  hitcount: 256
rank_at 10  hitcount: 1314
rank_at 15  hitcount: 3843
rank_at 20  hitcount: 6959

 2m-ml
Full data #row:  1974692
rank_at 1  hitcount: 184
rank_at 5  hitcount: 1900
rank_at 10  hitcount: 6933
rank_at 15  hitcount: 9426
rank_at 20  hitcount: 17776

 ml_0.7_u_above_5
Full data #row:  707447
rank_at 1  hitcount: 14
rank_at 5  hitcount: 1777
rank_at 10  hitcount: 3853
rank_at 15  hitcount: 5296
rank_at 20  hitcount: 7063

 ml_0.7_u_above_5_3_r_thres
Full data #row:  574132
rank_at 1  hitcount: 275
rank_at 5  hitcount: 2690
rank_at 10  hitcount: 4016
rank_at 15  hitcount: 5979
rank_at 20  hitcount: 8040

 2m-ml_3_r_thres
Full data #row:  1614609
rank_at 1  hitcount: 3483
rank_at 5  hitcount: 5015
rank_

In [28]:
pop_results_old = pd.read_pickle('Results/BPR/pop_rank_hits')

FileNotFoundError: [Errno 2] No such file or directory: 'Results/BPR/pop_rank_hits'

In [ ]:
pop_results_old['ml_0.7_u_above_5_3_r_thres'] = pop_results['ml_0.7_u_above_5_3_r_thres']

In [30]:
pop_results.to_pickle('Results/BPR/pop_rank_hits')

In [ ]:
pop_results_old

In [ ]:
len(df)